In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import pytesseract
from collections import deque
import pytesseract

# Set the tesseract_cmd to the full path if it's not in the PATH
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"



# Initialize points storage for different colors
bpoints = [deque(maxlen=1024)]
gpoints = [deque(maxlen=1024)]
rpoints = [deque(maxlen=1024)]
ypoints = [deque(maxlen=1024)]

# Initialize the color index and other drawing variables
blue_index, green_index, red_index, yellow_index = 0, 0, 0, 0
kernel = np.ones((5, 5), np.uint8)  # Kernel for dilation
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (0, 255, 255)]  # BGR
colorIndex = 0  # Default color
paintWindow = np.ones((471, 636, 3)) * 255  # Canvas initialization
undo_stack = []  # Stack for undo functionality

# Mediapipe Hand Tracking
mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mpDraw = mp.solutions.drawing_utils

# Webcam Capture
cap = cv2.VideoCapture(0)

# OCR Configuration for English and Math symbols
custom_config = r'--psm 6 --oem 3 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz '

# Flag for text detection
detect_text = False

# Function to draw the buttons for color and other actions
def draw_button(frame):
    cv2.rectangle(frame, (650, 50, 200, 60), (255, 0, 0), 2)
    cv2.putText(frame, "Detect Text", (660, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

# Save the current state of the drawing for undo functionality
def save_state():
    undo_stack.append(paintWindow.copy())

# Reset points for a new drawing
def reset_points():
    global bpoints, gpoints, rpoints, ypoints, blue_index, green_index, red_index, yellow_index
    bpoints = [deque(maxlen=1024)]
    gpoints = [deque(maxlen=1024)]
    rpoints = [deque(maxlen=1024)]
    ypoints = [deque(maxlen=1024)]
    blue_index = green_index = red_index = yellow_index = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # Flip frame horizontally
    framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Draw control buttons
    frame = cv2.rectangle(frame, (40, 1), (140, 65), (0, 0, 0), 2)
    frame = cv2.rectangle(frame, (160, 1), (255, 65), (255, 0, 0), 2)
    frame = cv2.rectangle(frame, (275, 1), (370, 65), (0, 255, 0), 2)
    frame = cv2.rectangle(frame, (390, 1), (485, 65), (0, 0, 255), 2)
    frame = cv2.rectangle(frame, (505, 1), (600, 65), (0, 255, 255), 2)
    cv2.putText(frame, "CLEAR", (49, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "BLUE", (185, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "GREEN", (298, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "RED", (420, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    cv2.putText(frame, "YELLOW", (520, 33), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 2, cv2.LINE_AA)
    draw_button(frame)  # Draw OCR button

    # Process hand landmarks
    result = hands.process(framergb)
    if result.multi_hand_landmarks:
        landmarks = []
        for handslms in result.multi_hand_landmarks:
            for lm in handslms.landmark:
                lmx = int(lm.x * 640)
                lmy = int(lm.y * 480)
                landmarks.append([lmx, lmy])

            mpDraw.draw_landmarks(frame, handslms, mpHands.HAND_CONNECTIONS)

        fore_finger = (landmarks[8][0], landmarks[8][1])
        center = fore_finger
        thumb = (landmarks[4][0], landmarks[4][1])
        cv2.circle(frame, center, 3, (0, 255, 0), -1)

        if (thumb[1] - center[1] < 30):
            bpoints.append(deque(maxlen=512))
            blue_index += 1
            gpoints.append(deque(maxlen=512))
            green_index += 1
            rpoints.append(deque(maxlen=512))
            red_index += 1
            ypoints.append(deque(maxlen=512))
            yellow_index += 1

        elif center[1] <= 65:
            if 40 <= center[0] <= 140:  # Clear Button
                reset_points()
                paintWindow[67:, :, :] = 255
            elif 160 <= center[0] <= 255:
                colorIndex = 0  # Blue
            elif 275 <= center[0] <= 370:
                colorIndex = 1  # Green
            elif 390 <= center[0] <= 485:
                colorIndex = 2  # Red
            elif 505 <= center[0] <= 600:
                colorIndex = 3  # Yellow
        else:
            if colorIndex == 0:
                bpoints[blue_index].appendleft(center)
            elif colorIndex == 1:
                gpoints[green_index].appendleft(center)
            elif colorIndex == 2:
                rpoints[red_index].appendleft(center)
            elif colorIndex == 3:
                ypoints[yellow_index].appendleft(center)

    # Draw points on the canvas
    points = [bpoints, gpoints, rpoints, ypoints]
    for i in range(len(points)):
        for j in range(len(points[i])):
            for k in range(1, len(points[i][j])):
                if points[i][j][k - 1] is None or points[i][j][k] is None:
                    continue
                cv2.line(frame, points[i][j][k - 1], points[i][j][k], colors[i], 2)
                cv2.line(paintWindow, points[i][j][k - 1], points[i][j][k], colors[i], 2)
# Capture the ROI for OCR (Region of Interest)
    roi = paintWindow[67:, :]  # Area where drawing is happening
    roi = cv2.resize(roi, (640, 480))  # Resize for faster processing
    roi = np.uint8(roi)
    gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)  # Convert to grayscale

# Apply adaptive thresholding and dilation for better OCR
    adaptive_thresh = cv2.adaptiveThreshold(gray_roi, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)

# Apply dilation and erosion to make the text more continuous
    dilated_roi = cv2.dilate(adaptive_thresh, kernel, iterations=1)
    eroded_roi = cv2.erode(dilated_roi, kernel, iterations=1)

# Optional: Show the processed image for debugging
    cv2.imshow("Processed ROI", eroded_roi)

# OCR detection
    if detect_text:
        text = pytesseract.image_to_string(eroded_roi, config=custom_config)
        print("Detected Text:", text.strip())


    # Display the frames
    cv2.imshow("Paint", paintWindow)
    cv2.imshow("Drawing", frame)

    # Key handling
    key = cv2.waitKey(1)
    if key == ord('q'):  # Quit the program
        break
    elif key == ord('d'):  # Toggle OCR detection
        detect_text = not detect_text
    elif key == ord('z'):  # Undo drawing
        if undo_stack:
            paintWindow = undo_stack.pop()  # Restore the last saved state
            reset_points()  # Reset the drawing points for proper handling
            print("Undo successful!")

    # Save the current state for undo functionality
    save_state()
    

cap.release()
cv2.destroyAllWindows()
